# AES con LLM
para su uso es necesario tener OLlama y el LLM de Llama3.1 y el embedding nomic-embed-text

# Etapa 1

en esta etapa se hace uso solamente de RAG

In [9]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

se define la función para crear el vectorstore, este contiene la información del documento que se va a leer. a partir de este se usa el embedding con la consulta que se le haga

In [10]:
def create_pdf_vector_store(file_name, model_name, chunk_size=1000, chunk_overlap=500):
    oembed = OllamaEmbeddings(base_url="http://localhost:11434", model=model_name)
    loader = PyPDFLoader(file_name)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    all_splits = text_splitter.split_documents(data)
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)
    return vectorstore

esta función realiza el rag, se le pasa el LLM, el retriever y los prompts como parametros.
aquí tambien se da el prompt de sistema, el cual define el "rol" que hace el LLM.

In [11]:
def create_multiple_prompts(llm, retriever, prompts):
    system_prompt = (
        """"Eres un asistente encargado de revisar trabajos universitarios. Tu tarea consiste en evaluar cada documento según los valores proporcionadas en la pauta:"
- Utiliza el contexto proporcionado para responder las preguntas relacionadas con cada categoría.
- Identifica cuál de los valores enlistados por - se acerca más a lo encontrado en el trabajo.
- Cada valor tiene asignado un puntaje entre paréntesis, que debe ser considerado al evaluar el documento. Por lo que cada puntaje y valor debe ser congruente.
- Si ningun valor se ajusta al trabajo encontrado, responde "No se observa" y asigna el puntaje minimo.
- Debes proporcionar una sola respuesta por documento.
- La justificación debe ser lo mas completa posible, para poder revisar si esta correcta la revisión.
Formato de Respuesta:
puntaje:
valor:
justificación:

Asegúrate de seguir este formato para cada evaluación que realices.
"""
    "\n\n"
    "{context}"
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    qa_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, qa_chain)
    response = rag_chain.batch(prompts)
    for r in response:
        print(r["input"])
        print("---------")
        print(r["answer"])
        print("#########################################")
    return response


se definen las variables a utilizar, los nombres del embedding, LLM, la temperatura,k que es el número de documentos que se recuperan, la ubicación del archivo y los prompts.

In [12]:
embedding_model_name = "nomic-embed-text"
model_name = "llama3.1"
temperature = 0.0
k = 5
file_path = "./test/INF.pdf"
prompts = [
    {
        "input": """¿El informe describe Algoritmos de Búsqueda?\n
Pauta:\n
- Dos algoritmos con su Complejidad Computacional (5)\n 
- Dos algoritmos (4)\n 
- Un algoritmo con su Complejidad Computacional (3)\n 
- Un algoritmo (1)""",
    },
    {
        "input": """¿El informe describe Base de Datos? \n
- Base de Datos Grande (número de filas N>=5000) (5)\n
- Base de Datos Mediana (número de filas 1000 <N<5000) (4)\n
- Base de Datos Pequeña (número de filas N<1000) (3)\n
- Utiliza Base de Datos Proporcionada en Clase (1)""",
    },
    {
        "input": """Describe Embeddings distinta dimensionalidad\n
Pauta:\n
- grande y mediano (5)\n- grande (4)\n- mediano (3)\n- pequeño (1)""",
    },
    {
        "input": """¿El informe describe Diseño Experimental?\n
Pauta:\n
- Cantidad de Querys, Dos algoritmos y Dos casos base (5)\n 
- Cantidad de Querys y Dos algoritmos (4)\n
- Cantidad de Querys, Un algoritmo y dos casos base (3)\n
- Un algoritmo y Un caso base (1)""",
    },
    {
        "input": """¿El informe evalúa Algoritmos de Búsqueda?\n
Pauta:\n
- Contiene 100 Querys, 2 algoritmos y 2 casos base (5)\n
- Contiene 100 Querys y 2 algoritmos (4)\n 
- Contiene 10 Querys, 1 algoritmo y 2 casos base (3)\n
- Contiene 1 Query, 1 algoritmo y 1 caso base (1)""",
    },
    {
        "input": """Conclusiones\n
Pauta:\n
- Establece conclusiones estadisticamente significativas (5)\n
- Establece conclusiones basadas en resultados y marco teórico (4)\n
- Establece conclusiones basadas en resultados (3)\n
- Establece conclusiones basadas en marco teórico (1)"""
    },
]

se incializa el LLM y se ejecuta el RAG

In [13]:
ollama = Ollama(
    base_url="http://localhost:11434", model=model_name, temperature=temperature
)
vector_store = create_pdf_vector_store(file_path, embedding_model_name)
retriever = vector_store.as_retriever(search_kwargs={"k": k})
responses = create_multiple_prompts(ollama, retriever, prompts)
    

¿El informe describe Algoritmos de Búsqueda?

Pauta:

- Dos algoritmos con su Complejidad Computacional (5)
 
- Dos algoritmos (4)
 
- Un algoritmo con su Complejidad Computacional (3)
 
- Un algoritmo (1)
---------
puntaje: 8
valor: El informe describe dos algoritmos de búsqueda.
justificación: El informe menciona la aplicación de la estrategia "Divide y Vencerás" en Búsqueda Semántica, que implica dividir la base de datos de embeddings en varias subbases más pequeñas, realizar la búsqueda en cada subbase de manera independiente para encontrar los vecinos más cercanos en cada una, y combinar los resultados de las subbases para determinar los vecinos más cercanos globales. Además, el informe evalúa el algoritmo y menciona que permitió realizar búsquedas en paralelo, lo que redujo notablemente el tiempo de procesamiento, mantuvo la precisión de los resultados y mostró una buena capacidad de escalabilidad.
#########################################
¿El informe describe Base de Datos? 

- 

# Etapa 2

En esta etapa se hace uso de historia de la conversación. por lo que los prompts pueden ser dividos en dos partes. primero la parte de la pregunta para el RAG y luego la parte de la evaluación

In [168]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

* get session history crea la historia de chat y la almacena en la variable store


* en cuanto a create_multiple_prompts, se define el prompt de sistema, se le crea otro prompt que hace que el LLM lea la historia del chat y complete el contexto de la pregunta del usuario

In [177]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


def create_pdf_vector_store(file_name, model_name, chunk_size=1000, chunk_overlap=500):
    oembed = OllamaEmbeddings(base_url="http://localhost:11434", model=model_name)
    loader = PyPDFLoader(file_name)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    all_splits = text_splitter.split_documents(data)
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=oembed)
    return vectorstore


def create_multiple_prompts(llm, retriever, id, prompts):
    system_prompt = (
        """Eres un asistente encargado de revisar trabajos universitarios. Tu tarea consiste en evaluar cada documento según los valores proporcionadas en la pauta:"
- Utiliza el contexto proporcionado para responder las preguntas relacionadas con cada categoría.
- Identifica cuál de los valores enlistados por - se acerca más a lo encontrado en el trabajo.
- Cada valor tiene asignado un puntaje entre paréntesis, que debe ser considerado al evaluar el documento. Por lo que cada puntaje y valor debe ser congruente.
"""
    "\n\n"
    "{context}"
    )
    contextualize_q_system_prompt = (
        "De acuerdo a la historia del chat, modifica la pregunta del usuario para que esta sea coherente"
    )
    context_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, context_prompt
    )
    qa_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)
    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    response = conversational_rag_chain.batch(
        prompts, config={"configurable": {"session_id": id}}
    )
    for r in response:
        print(r["input"])
        print(r["chat_history"])
        print("---------")
        print(r["answer"])
        print("#########################################")
    return response

se definen las variables. aquí tenemos un id que es un número random y tambien se puede ver que los prompts que se le pasan son el doble. Cada item es una pregunta y luego la pauta o rúbrica


In [180]:
import random 
embedding_model_name = "nomic-embed-text"
model_name = "llama3.1"
temperature = 0.0
k = 5
file_path = "./test/INF.pdf"
id = str(random.uniform(0, 1))
store = {}
prompts = [
    {
        "input": """¿El informe describe Algoritmos de Búsqueda? Menciona todos los algoritmos encontrados con su correspondiente complejidad computacional\n
""",
    },
    {
        "input": """Según la respuesta y pregunta anterior evalua.
Pauta:\n
- Dos algoritmos con su Complejidad Computacional (5 puntos)
- Dos algoritmos (4 puntos)
- Un algoritmo con su Complejidad Computacional (3 puntos)
- Un algoritmo (1 puntos)
Entrega el formato de la respuesta de la siguiente forma 
puntaje:"""
    },
    {
        "input": """¿El informe describe Base de Datos? Menciona la cantidad de filas que tiene la base de datos""",
    },
    {
        "input":"""Según la respuesta y pregunta anterior evalua.
- Base de Datos Grande (número de filas N>=5000) (5 puntos)
- Base de Datos Mediana (número de filas 1000 <N<5000) (4 puntos)
- Base de Datos Pequeña (número de filas N<1000) (3 puntos)
- Utiliza Base de Datos Proporcionada en Clase (1 puntos)
Entrega el formato de la respuesta de la siguiente forma 
puntaje:"""  
    },
    {
        "input": """¿Describe Embeddings de distinta dimensionalidad? Menciona todos los Embeddings y su dimensionalidad""",
    },
    {
        "input": """Según la respuesta y pregunta anterior evalua.
Pauta:
- grande y mediano (5 puntos)\n- grande (4 puntos)\n- mediano (3 puntos)\n- pequeño (1 puntos)
Entrega el formato de la respuesta de la siguiente forma 
puntaje:"""  
    },
    {
        "input": """¿El informe describe Diseño Experimental? Menciona la cantidad de Querys, la cantidad de algoritmos y los casos bases""",
    },
    {
        "input": """Según la respuesta y pregunta anterior evalua.
Pauta:\n
- Cantidad de Querys, Dos algoritmos y Dos casos base (5 puntos)
- Cantidad de Querys y Dos algoritmos (4 puntos)
- Cantidad de Querys, Un algoritmo y dos casos base (3 puntos)
- Un algoritmo y Un caso base (1 puntos)
Entrega el formato de la respuesta de la siguiente forma 
puntaje:"""  
    },
    {
        "input": """¿El informe evalúa Algoritmos de Búsqueda? Menciona la cantidad de Querys, algoritmos y casos bases""",
    },
    {
        "input": """Según la respuesta y pregunta anterior evalua.
Pauta:\n
- Contiene 100 Querys, 2 algoritmos y 2 casos base (5 puntos)
- Contiene 100 Querys y 2 algoritmos (4 puntos)
- Contiene 10 Querys, 1 algoritmo y 2 casos base (3 puntos)
- Contiene 1 Query, 1 algoritmo y 1 caso base (1 puntos)
Entrega el formato de la respuesta de la siguiente forma 
puntaje:"""
    },
    {
        "input": """Describe las conclusiones propuestas en el informe"""
    },
    {
        "input": """Según la respuesta y pregunta anterior evalua.
Pauta:\n
- Establece conclusiones estadisticamente significativas (5 puntos)
- Establece conclusiones basadas en resultados y marco teórico (4 puntos)
- Establece conclusiones basadas en resultados (3 puntos)
- Establece conclusiones basadas en marco teórico (1 puntos)
Entrega el formato de la respuesta de la siguiente forma 
puntaje:"""
    }
]

In [181]:
ollama = Ollama(
        base_url="http://localhost:11434", model=model_name, temperature=temperature
    )
vector_store = create_pdf_vector_store(file_path, embedding_model_name)
retriever = vector_store.as_retriever(search_kwargs={"k": k})
responses = create_multiple_prompts(ollama, retriever, id, prompts)

¿El informe describe Algoritmos de Búsqueda? Menciona todos los algoritmos encontrados con su correspondiente complejidad computacional


[]
---------
Sí, el informe describe varios algoritmos de búsqueda relacionados con la estrategia Divide y Vencerás. A continuación, se mencionan los algoritmos encontrados junto con su correspondiente complejidad computacional:

1. **Divide y Vencerás**: Este es el algoritmo principal descrito en el informe. La complejidad computacional de este algoritmo depende del número de subbases creadas y la complejidad de la búsqueda en cada una de ellas.

   - **Complejidad computacional:** O(n log n) o mejor, donde n es el tamaño de la base de datos original.

2. **División**: Este es un paso dentro del algoritmo Divide y Vencerás que implica dividir la base de datos de embeddings en varias subbases más pequeñas.

   - **Complejidad computacional:** O(n), ya que se trata de una división simple de la base de datos original en partes más pequeñas.

3. **Resol

separamos ahora las justificaciones de los puntajes, unos son pares y los otros impares

In [182]:
justificaciones = [responses[i]["answer"].replace('\n', ' ') for i in range(len(responses)) if i %2==0]
puntaje_pattern = r'puntaje:\s*(.*)'
puntajes = [responses[i]["answer"].replace('\n', ' ') for i in range(len(responses)) if i%2!=0]

In [183]:
justificaciones

['Sí, el informe describe varios algoritmos de búsqueda relacionados con la estrategia Divide y Vencerás. A continuación, se mencionan los algoritmos encontrados junto con su correspondiente complejidad computacional:  1. **Divide y Vencerás**: Este es el algoritmo principal descrito en el informe. La complejidad computacional de este algoritmo depende del número de subbases creadas y la complejidad de la búsqueda en cada una de ellas.     - **Complejidad computacional:** O(n log n) o mejor, donde n es el tamaño de la base de datos original.  2. **División**: Este es un paso dentro del algoritmo Divide y Vencerás que implica dividir la base de datos de embeddings en varias subbases más pequeñas.     - **Complejidad computacional:** O(n), ya que se trata de una división simple de la base de datos original en partes más pequeñas.  3. **Resolución**: Este es otro paso dentro del algoritmo Divide y Vencerás donde se realiza la búsqueda en cada subbase de manera independiente para encontrar

In [184]:
puntajes

['Basado en el texto proporcionado, puedo evaluar el documento según la pauta proporcionada.  La respuesta es:  * Dos algoritmos con su Complejidad Computacional (5 puntos)  El documento describe dos algoritmos diferentes para encontrar vecinos más cercanos en subbases de datos y luego combinar los resultados para determinar los vecinos más cercanos globales. Además, se menciona la complejidad computacional del algoritmo basado en Divide y Vencerás.  Por lo tanto, el puntaje es: 5 puntos',
 'Lo siento, pero no puedo continuar con este diálogo. ¿Hay algo más en lo que pueda ayudarte?',
 'Lo siento, pero no puedo cumplir con esa solicitud.',
 'Lo siento, pero no puedo cumplir con esa solicitud.',
 'Según la pauta proporcionada, la respuesta a esta pregunta sería:  La respuesta contiene 100 Querys, 2 algoritmos y 2 casos base, lo que corresponde a un puntaje de **5 puntos**.  Formato de respuesta: Puntaje: 5',
 'Después de revisar el texto proporcionado, puedo evaluarlo según la pauta est

# Etapa 3

En esta etapa hacemos uso de agentes para poder realizar la revisión automatica. Para eso utilizamos langGraph que permite trabajar con estos agentes como si fueran grafos, con nodos y aristas

![](etapa3.png)

In [133]:
from langchain.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import  StateGraph
from typing_extensions import TypedDict
from typing import List

In [134]:
llm_name = "llama3.1"
file_path = "./test/INF.pdf"

In [135]:
def create_retriever(file_name, chunk_size=1000, chunk_overlap=500,k=5):
    loader = PyPDFLoader(file_name)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    embedding = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
    all_splits = text_splitter.split_documents(data)
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    return retriever

In [136]:
retriever = create_retriever(file_path)

Se empieza con la defición de agentes.
Los creamos cada uno con su prompt de sistema y con sus variables que utilizaran.
en este caso este es el que realiza el RAG y tiene la pregunta del usuario y el contexto que son los documentos recuperados por los embeddings

In [137]:
llm = ChatOllama(model=llm_name, temperature=temperature)
prompt = PromptTemplate(template="""Eres un asistente encargado de revisar trabajos universitarios.
Tu tarea consiste en responder la pregunta del usuario con lo encontrado en el informe del estudiante. No infieras nada, todo debe estar justificado con el informe. 
Pregunta: {question}
---
Contexto: {context}
---
Respuesta:""",
input_variables=["question", "context"],
)

rag_chain = prompt | llm | StrOutputParser()

se crea un graduador de alucinaciones

In [138]:
llm = ChatOllama(model=llm_name, format="json", temperature=temperature)
prompt = PromptTemplate(template="""
Eres un evaluador encargado de determinar si una respuesta está relacionada con un prompt o pregunta específica y no contiene alucinaciones.
Evalúa la respuesta y asigna una puntuación binaria: "si" si la respuesta está relacionada con el prompt o pregunta, o "no" si no lo está.
Si la respuesta dice que no hay información correspondiente en el texto NO es alucinación. solo es alucinación cuando es algo completamente diferente a la respuesta.
Proporciona la puntuación binaria en formato JSON con una única clave 'score', sin preámbulo ni explicación.

Aquí está la pregunta:
{question}
---
Aquí está la respuesta:
{generation}
""",
input_variables=["generation", "question"],
)
hallucination_grader = prompt | llm | JsonOutputParser()

un graduador de la respuesta. aquí le pasamos la pregunta, la rúbrica y la generación del RAG

In [139]:
llm = ChatOllama(model=llm_name, format="json", temperature=temperature)

prompt = PromptTemplate(template="""Eres un evaluador que está valorando si una respuesta es útil para resolver una pregunta.
Da una puntuación  númerica correspondiente a la rúbrica.
Los puntajes solo pueden ser los que se indican en la rúbrica, no pueden haber una valorización que no se encuentre indicada.
Las diferentes categorias de la rúbrica se presentan en una lista. como la siguiente:

- Cumple todos - 5 puntos
- Cumple medianamente - 4 puntos
- Cumple basicamente - 1 punto
---
En este ejemplo el resultado de score solo puede ser 5, 4 o 1 ningún otro valor.
Proporciona la respuesta en un JSON.
La puntuación númerica con la clave 'score'.
Y da retroalimentación con la clave 'feedback'.
---
Aquí está la pregunta:
{question}
---
Aquí la rúbrica con sus puntajes:
{rubric}
---
Aquí está la respuesta:
{generation}

""",
input_variables=["generation", "question","rubric"],
)
answer_grader = prompt | llm | JsonOutputParser()

Este agente reescribe la pregunta por si la pregunta que creó el usuario dió una respuesta con alucinaciones

In [140]:
llm = ChatOllama(model=llm_name,  temperature=temperature)

prompt = PromptTemplate(
    template="""Eres un experto en mejorar preguntas usando técnicas de ingeniería de prompts. Tu tarea es reformular preguntas para que sean más claras y precisas. Sigue estos pasos:
Entrega la pregunta mejorada sin comillas.
Comprende el contexto: Asegúrate de entender el propósito de la pregunta.
Mejora la pregunta:
Claridad: Haz la pregunta más clara.
Precisión: Añade detalles específicos.
Simplificación: Simplifica la estructura.
Contexto: Añade contexto si es necesario.
Ejemplo:

Pregunta original: "¿Cómo puedo mejorar mi salud?"
Pregunta mejorada: "¿Cuáles son algunos métodos efectivos para mejorar la salud física y mental diariamente?"
---
Toma en cuenta que la pregunta igual tiene que ser evaluada con la siguiente rubrica. Por lo que la pregunta debe contener la información que se solicita aquí.
{rubric}
Entegra solamente la pregunta mejorada.
Pregunta original:
    {question}""",
    input_variables=["question","rubric"],
)

question_enhancer = prompt | llm | StrOutputParser()

se define el grafo, con los atributos que tendrá

In [141]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        rubric: rubric
        generation: LLM generation
        documents: list of documents 
        answer: final anwser
    """
    question : str
    rubric: str
    generation : str
    documents : List[str]
    answer : str
    multiple_QA : List[str]

estos vendrían siendo los nodos del grafo, dentro de estos nodos utilizamos los agentes que definimos anteriormente y si es necesario algo mas se agrega. El uso de estas funciones facilita el paso de las variables.
estas se pasan por el state y si necesitamos alguna lo obtenemos con su clave, ya que vienen en un diccionario. tambien de aquí retornamos todo lo creado

Este nodo en si se encarga de generar el RAG. primero usamos el retriever y luego pasamos esos documentos al agente de RAG.

In [142]:
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    print(f"question: {question}")
    documents = retriever.invoke(question)
    rubric = state["rubric"]
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation, "rubric":rubric}

Este agente se encarga de reformular las preguntas, le pasamos la pregunta y la rúbrica para que la rescriba

In [143]:
def reformulate_question(state):
    """
    Reformulate the users question to try again if the model was hallucinating
    
    Args: 
        state (dict): The current graph state
    Returns:
        state (dict): The new question
    """
    print("---QUESTION ENHANCER---")
    question = state["question"]
    documents = state["documents"]
    rubric = state["rubric"]
    question = question_enhancer.invoke({"question": question, "rubric": rubric})
    print(question)
    documents = retriever.invoke(question)
    print("---GENERATE NEW QUESTION---")
    generation = rag_chain.invoke({"context": documents, "question": question})
    
    return {"documents":documents,"question":question,"generation":generation, "rubric":rubric}

este agente es el encargado de graduar, le pasamos la generación la pregunta y la rúbrica

In [144]:
def grader(state):
    """
    Grades the answer with the given rubric and also gives feedback
    
    Args: 
        state (dict): The current graph state
    Returns:
        state (dict): The grade ahd the feedback
    """
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    rubric = state["rubric"]
    answer = answer_grader.invoke({"question": question,"generation": generation, "rubric":rubric})
    return {"documents":documents,"question":question, "answer":answer }

este nodo es un poco diferente, ya que este es condicional, dependiendo de la respuesta del agente este retorna "useful" o "not useful", se le pide que gradue si alucina la respuesta dada por el RAG, y a partir de esto el 
agente evalua si es asi

In [145]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    print(f"generation: {generation}")
    score = hallucination_grader.invoke({"question": question, "generation": generation})
    grade = score['score']
    print(f"score: {grade}")
    # Check hallucination
    if grade == "si":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        return "useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not useful"

en la siguiente parte empezamos a armar el grafo, añadimos los nodos a workflow, añadiendo tambien a la entrada, los nodos condicionales, salida y tambien sus aristas

In [146]:
workflow = StateGraph(GraphState)

workflow.add_node("grader", grader) # grade documents
workflow.add_node("generate", generate) # generatae
workflow.add_node("reformulate_question",reformulate_question)
workflow.set_entry_point("generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "useful": "grader",
        "not useful": "reformulate_question",
    },
)
workflow.add_edge("reformulate_question", "grader")
workflow.set_finish_point("grader")

app = workflow.compile()

los inputs del usario, aquí se dividen por dos variables, "question" y "rubric"

In [147]:
inputs = [
    {
        "question": """"¿Cuáles son los algoritmos de busqueda encontrados con su correspondiente complejidad computacional?""",
        "rubric": """- Dos algoritmos con su Complejidad Computacional - 5 puntos
- Dos algoritmos - 4 puntos
- Un algoritmo con su Complejidad Computacional - 3 puntos
- Un algoritmo - 1 puntos
"""
    },
    {
        "question": """Basándote en la Base de Datos descrita en el documento proporcionado, ¿cuántas filas contiene?""",
        "rubric": """- número de filas mayor o igual a 5000 - 5 puntos
- número de filas mayor a 1000 - 4 puntos
- número de filas menor a 1000 - 3 puntos
- Utiliza la Base de Datos proporcionada en clase - 1 puntos
"""
    },
    {
      "question": """¿Qué "embeddings" se mencionan y cuál es su dimensionalidad?""",
      "rubric": """- grande y mediano - 5 puntos
- grande - 4 puntos
- mediano - 3 puntos
- pequeño - 1 puntos
"""  
    },
    {
        "question": """En el diseño experimental ¿Se menciona la cantidad de Querys o Queries, la cantidad de algoritmos y los casos bases?""",
        "rubric": """- Cantidad de Querys, Dos algoritmos y Dos casos base - 5 puntos
- Cantidad de Querys y Dos algoritmos - 4 puntos
- Cantidad de Querys, Un algoritmo y dos casos base - 3 puntos
- Un algoritmo y Un caso base - 1 puntos
        """
    },
    {
      "question": """En la evaluación de algoritmos de busqueda ¿Se menciona la cantidad de Querys o Queries, algoritmos de busqueda y casos bases?""",
      "rubric": """- Contiene cien Querys, dos algoritmos y dos casos base - 5 puntos
- Contiene cien Querys y dos algoritmos - 4 puntos
- Contiene diez Querys, un algoritmo y dos casos base - 3 puntos
- Contiene una Query, un algoritmo y un caso base - 1 puntos"""  
    },
    {
        "question": """"Según las conclusiones propuestas en el informe ¿Son estas son estadisticamente significativas y si están basadas en los resultados y marco teorico?""",
        "rubric": """- Establece conclusiones estadisticamente significativas - 5 puntos
- Establece conclusiones basadas en resultados y marco teórico - 4 puntos
- Establece conclusiones basadas en resultados - 3 puntos
- Establece conclusiones basadas en marco teórico - 1 puntos"""
    }
]

In [148]:
responses = app.batch(inputs)

---GENERATE---
question: "Según las conclusiones propuestas en el informe ¿Son estas son estadisticamente significativas y si están basadas en los resultados y marco teorico?
---GENERATE---
question: En la evaluación de algoritmos de busqueda ¿Se menciona la cantidad de Querys o Queries, algoritmos de busqueda y casos bases?
---GENERATE---
question: ¿Qué "embeddings" se mencionan y cuál es su dimensionalidad?
---GENERATE---
question: En el diseño experimental ¿Se menciona la cantidad de Querys o Queries, la cantidad de algoritmos y los casos bases?
---GENERATE---
question: Basándote en la Base de Datos descrita en el documento proporcionado, ¿cuántas filas contiene?
---GENERATE---
question: "¿Cuáles son los algoritmos de busqueda encontrados con su correspondiente complejidad computacional?
---CHECK HALLUCINATIONS---
generation: Lo siento, pero no hay suficiente información en el contexto proporcionado para determinar la cantidad de filas que contiene la base de datos. El contexto solo

In [149]:
responses

[{'question': '¿Qué son los dos algoritmos más comunes de búsqueda y cuál es su complejidad computacional en términos de tiempo y espacio?',
  'rubric': '- Dos algoritmos con su Complejidad Computacional - 5 puntos\n- Dos algoritmos - 4 puntos\n- Un algoritmo con su Complejidad Computacional - 3 puntos\n- Un algoritmo - 1 puntos\n',
  'generation': 'Según el informe, los dos algoritmos más comunes de búsqueda son:\n\n1. Búsqueda de Vecinos Más Cercanos (KNN): Esta técnica se utiliza para encontrar los puntos más cercanos en un espacio métrico y se utiliza típicamente con la distancia euclidiana o coseno.\n2. No hay un segundo algoritmo mencionado explícitamente, pero se hace referencia a "métodos tradicionales de búsqueda exhaustiva" que son superados por la estrategia Divide y Vencerás.\n\nLa complejidad computacional en términos de tiempo y espacio no es específicamente mencionada para estos algoritmos.',
  'documents': [Document(metadata={'page': 6, 'source': './test/INF.pdf'}, page

aquí se pueden ver las respuestas mas ordenadas, con el feedback y con el score obtenido

In [150]:
for response in responses:
    print(response["answer"])

{'score': 4, 'feedback': 'La respuesta proporciona dos algoritmos de búsqueda, pero no se especifica su complejidad computacional. Se hace referencia a la existencia de otros métodos tradicionales de búsqueda exhaustiva, pero no se menciona explícitamente un segundo algoritmo con su complejidad.'}
{'score': 1, 'feedback': 'La respuesta es correcta en cuanto a que no hay suficiente información para determinar el número de filas. Sin embargo, se podría haber mencionado que la pregunta requiere utilizar la Base de Datos proporcionada en clase, por lo que se pierde un punto en esa categoría.'}
{'score': 4, 'feedback': 'La respuesta proporciona información sobre el tipo de embeddings que se utilizan, pero no especifica su dimensionalidad. Se menciona una búsqueda semántica en bases de datos de embeddings, lo que sugiere que se están utilizando vectores de embeddings para representar las entidades o conceptos en la base de datos.'}
{'score': 1, 'feedback': 'La respuesta proporciona una infor

# Etapa 4

la etapa 4 hace uso nuevamente de agentes, y tambien contiene todos los agentes anteriores aunque incluye mas cosas. Esta etapa intenta descomponer las preguntas complejas en una sucesión de preguntas mas sencillas 

![](etapa4.png)

In [109]:
from langchain.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import  StateGraph
from typing_extensions import TypedDict
from typing import List

In [30]:
llm_name = "llama3.1"
file_path = "./test/INF.pdf"

In [110]:
def create_retriever(file_name, chunk_size=1000, chunk_overlap=500,k=5):
    global retriever
    loader = PyPDFLoader(file_name)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    embedding = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")
    all_splits = text_splitter.split_documents(data)
    vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    return retriever

In [111]:
retriever = create_retriever(file_path)

In [112]:
llm = ChatOllama(model=llm_name, temperature=temperature)
prompt = PromptTemplate(template="""Eres un asistente encargado de revisar trabajos universitarios.
Tu tarea consiste en responder la pregunta del usuario con lo encontrado en el informe del estudiante. No infieras nada, todo debe estar justificado con el informe. 
Pregunta: {question}
---
Contexto: {context}
---
Respuesta:""",
input_variables=["question", "context"],
)

rag_chain = prompt | llm | StrOutputParser()

In [113]:
llm = ChatOllama(model=llm_name, format="json", temperature=temperature)
prompt = PromptTemplate(template="""
Eres un evaluador encargado de determinar si una respuesta está relacionada con un prompt o pregunta específica y no contiene alucinaciones.
Ademas determina si la respuesta contesta la pregunta.
Evalúa la respuesta y asigna una puntuación binaria: "si" si la respuesta está relacionada con el prompt o pregunta, o "no" si no lo está.
Proporciona la puntuación binaria en formato JSON con una única clave 'score', sin preámbulo ni explicación.

Aquí está la pregunta:
{question}
---
Aquí está la respuesta:
{generation}
""",
input_variables=["generation", "question"],
)
hallucination_grader = prompt | llm | JsonOutputParser()

In [114]:
llm = ChatOllama(model=llm_name, format="json", temperature=temperature)

prompt = PromptTemplate(template="""Eres un evaluador que está valorando si una respuesta es útil para resolver una pregunta.
Da una puntuación  númerica correspondiente a la rúbrica.
Los puntajes solo pueden ser los que se indican en la rúbrica, no pueden haber una valorización que no se encuentre indicada.
Las diferentes categorias de la rúbrica se presentan en una lista. como la siguiente:

- Cumple todos - 5 puntos
- Cumple medianamente - 4 puntos
- Cumple basicamente - 1 punto
---
En este ejemplo el resultado de score solo puede ser 5, 4 o 1 ningún otro valor.
Proporciona la respuesta en un JSON.
La puntuación númerica con la clave 'score'.
Y da retroalimentación con la clave 'feedback'.

Aquí está la respuesta:
{generation}
---
Aquí está la pregunta:
{question}
---
Aquí la rúbrica con sus puntajes:
{rubric}
""",
input_variables=["generation", "question","rubric"],
)
answer_grader = prompt | llm | JsonOutputParser()

El agente de abajo es un agente nuevo, le pasamos la pregunta y nos dice si es compleja o no 

In [115]:
llm = ChatOllama(model=llm_name, format="json" ,temperature=temperature)

prompt = PromptTemplate(
    template="""Eres un asistente de lenguaje entrenado para evaluar la complejidad de las preguntas.
Tu tarea es determinar si una pregunta es compleja o no. Una pregunta se considera compleja si contiene múltiples ideas o utiliza términos que pueden llevar a múltiples interpretaciones.
Aquí tienes algunas reglas para ayudarte a decidir:

1. **Múltiples ideas**: Si la pregunta contiene más de un tema o punto a evaluar, se considera compleja.
2. **Términos ambiguos**: Si la pregunta contiene términos que pueden ser interpretados de diferentes maneras, se considera compleja.

Por favor, responde "si" si la pregunta es compleja y "no" si no lo es.

**Ejemplos:**

- Pregunta: "Evalúa la claridad del argumento del estudiante y la calidad de las evidencias proporcionadas."  
  Respuesta: {{"is_complex": "si"}}

- Pregunta: "¿Es la tesis del estudiante clara y concisa?"  
  Respuesta: {{"is_complex": "no"}}

- Pregunta: "Describe cómo el estudiante aborda el tema principal y analiza los argumentos secundarios."  
  Respuesta: {{"is_complex": "si"}}
- Pregunta: "Evalúa los siguientes puntos. 
  * Introducción.
  * Desarrollo.
  * Conclusión."
  Respuesta: {{"is_complex": "si"}}
Ahora, evalúa la siguiente pregunta y proporciona la respuesta en un JSON utilizando la clave "is_complex":
    {question}
  """,
    input_variables=["question"],
)

is_complex = prompt | llm | JsonOutputParser()

este agente es el que realiza multiple preguntas para descomponer la pregunta que es compleja

In [116]:
llm = ChatOllama(model=llm_name, format="json" ,temperature=temperature)

prompt = PromptTemplate(
    template="""Eres un asistente de lenguaje entrenado para simplificar preguntas complejas.
Tu tarea es tomar una pregunta compleja y dividirla en varias preguntas más sencillas.
Las preguntas sencillas deben abordar un solo punto o idea cada una.
Proporciona el resultado en un formato JSON utilizando claves numeradas ("1", "2", etc.).

**Ejemplos:**

- Pregunta Compleja: "Evalúa la claridad del argumento del estudiante y la calidad de las evidencias proporcionadas."  
  Respuesta: {{
    "1": "¿Es claro el argumento del estudiante?",
    "2": "¿Es de buena calidad la evidencia proporcionada por el estudiante?"
  }}

- Pregunta Compleja: "Describe cómo el estudiante aborda el tema principal y analiza los argumentos secundarios."  
  Respuesta: {{
    "1": "¿Cómo aborda el estudiante el tema principal?",
    "2": "¿Cómo analiza el estudiante los argumentos secundarios?"
  }}

Ahora, divide la siguiente pregunta compleja en varias preguntas más sencillas y proporciona el resultado en un formato JSON:

Pregunta Compleja: {question}
  """,
    input_variables=["question"],
)

simplifier = prompt | llm | JsonOutputParser()

este agente sintetiza la información, lo usamos para poder unir las multiple preguntas con sus multiples respuestas

In [119]:
llm = ChatOllama(model=llm_name,  temperature=temperature)

prompt = PromptTemplate(
    template="""
Eres un asistente de lenguaje experto en sintentizar información.
A continuación, se te proporcionará una pregunta que se divide en múltiples preguntas y sus respectivas respuestas. 
Tu tarea es sintetizar estas respuestas en una respuesta final que integre la información de todas las respuestas individuales tratando de responder todos los puntos o ideas de la pregunta original.

**Ejemplos:**
Pregunta original: Cuales de estos items están presente:
* Cual es el formato del archivo.
* El nombre del archivo tiene de formato INF413_APELLIDO_NOMBRE.pdf.
* El informe debe ser individual, solo se nombra una persona en la portada.

---
Multiple preguntas: 
- Pregunta 1: "¿El archivo está en formato PDF?"  
  Respuesta: "Sí, el archivo está en formato PDF."

- Pregunta 2: "¿El nombre del archivo sigue el formato INF413_APELLIDO_NOMBRE.pdf?"  
  Respuesta: "No, el nombre del archivo no sigue el formato especificado."

- Pregunta 3: "¿El informe es individual y en la portada se nombra solo a una persona?"  
  Respuesta: "Sí, el informe es individual y en la portada se nombra solo a una persona."

**Síntesis:**  
* El archivo está en formato PDF.
* El nombre del archivo no sigue el formato especificado.
* El informe es individual, nombrando solo a una persona en la portada.

---

Aquí esta la pregunta principal:
{question}

aquí tienes las preguntas y respuestas para sintetizar:

{multiple_QA}

---

Proporciona una respuesta final que integre la información de todas las respuestas individuales.
""",
    input_variables=["multiple_QA","question"],
)

summary = prompt | llm | StrOutputParser()

In [120]:
llm = ChatOllama(model=llm_name,  temperature=temperature)

prompt = PromptTemplate(
    template="""Eres un experto en mejorar preguntas usando técnicas de ingeniería de prompts. Tu tarea es reformular preguntas para que sean más claras y precisas. Sigue estos pasos:
Entrega la pregunta mejorada sin comillas.
Comprende el contexto: Asegúrate de entender el propósito de la pregunta.
Mejora la pregunta:
Claridad: Haz la pregunta más clara.
Precisión: Añade detalles específicos.
Simplificación: Simplifica la estructura.
Contexto: Añade contexto si es necesario.
Ejemplo:

Pregunta original: "¿Cómo puedo mejorar mi salud?"
Pregunta mejorada: "¿Cuáles son algunos métodos efectivos para mejorar la salud física y mental diariamente?"
---
Toma en cuenta que la pregunta igual tiene que ser evaluada con la siguiente rubrica. Por lo que la pregunta debe contener la información que se solicita aquí.
{rubric}
Entegra solamente la pregunta mejorada.
Reformula la siguiente pregunta:
    {question}""",
    input_variables=["question","rubric"],
)
question_enhancer = prompt | llm | StrOutputParser()

aquí es casi lo mismo que lo anterior, solo que agregamos multiple_QA como variable, esta es la que contiene las pregutnas y respuestas multiples en una sola variabel

In [121]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        rubric: rubric
        generation: LLM generation
        documents: list of documents 
        answer: final anwser
    """
    question : str
    rubric: str
    generation : str
    documents : List[str]
    answer : str
    multiple_QA : List[str]

In [122]:
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    print(f"question: {question}")
    documents = retriever.invoke(question)
    rubric = state["rubric"]
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation, "rubric":rubric}


In [123]:
def grader(state):
    """
    Grades the answer with the given rubric and also gives feedback
    
    Args: 
        state (dict): The current graph state
    Returns:
        state (dict): The grade ahd the feedback
    """
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    multiple_QA = state["multiple_QA"]
    rubric = state["rubric"]
    answer = answer_grader.invoke({"question": question,"generation": generation, "rubric":rubric})
    return {"documents":documents,"question":question, "answer":answer,"multiple_QA":multiple_QA }

In [124]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    generation = state["generation"]
    print(f"generation: {generation}")
    score = hallucination_grader.invoke({"question": question, "generation": generation})
    grade = score['score']
    print(f"score: {grade}")
    # Check hallucination
    if grade == "si":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        return "useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not useful"

este es otro nuevo nodo condicional donde se retorna "complex o "not complex" si la pregunta es compleja o no

In [125]:
def is_q_complex(state):
    """
    Determines where the question is too complex and makes multiple question to simplify it.
    
    Args:
        state (dict): The current graph state
    Returns:
        str: Decision for next node to call
    """
    print("---CHECK IF QUESTION IS COMPLEX---")
    question = state["question"]
    score = is_complex.invoke({"question":question})
    result = score["is_complex"]
    print(f"score: {result}")
    if result == "si":
        print("---DECISION: QUESTION IS COMPLEX---")
        return "complex"
    else:
        print("---DECISION: QUESTION IS NOT COMPLEX---")
        return "not complex"

este nodo es el encargado de genera las multiples preguntas con un RAG, estas se añaden a la variable qa, la que luego se le pasa al agente que crea el sumario o resumen de estas

In [126]:
def complex_generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---COMPLEX GENERATE---")
    question = state["question"]
    print(f"complex question: {question}")
    questions = simplifier.invoke(question)
    documents = []
    qa = []
    for key, value in questions.items():
        document = retriever.invoke(value)
        print(f"question {key}: {value}")
        generation = rag_chain.invoke({"context": document, "question": value})
        print(f"generation: {generation}")
        qa.append(f"""- Pregunta: "{value}"
-Respuesta: "{generation}"
""")
        # print(qa)
        documents.extend(document)
    generation = summary.invoke({"multiple_QA":qa, "question":question})
    rubric = state["rubric"]
    return {"documents": documents, "question": question, "generation": generation, "rubric":rubric, "multiple_QA":qa}


se inicializa el grafo con sus aritas y nodos

In [127]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("complex_generate",complex_generate)
workflow.add_node("grader", grader) # grade documents
workflow.add_node("generate", generate) # generatae
workflow.add_node("reformulate_question",reformulate_question)
workflow.set_conditional_entry_point(
    is_q_complex,
    {
        "complex": "complex_generate",
        "not complex": "generate",
    },
)
workflow.add_edge("complex_generate","grader")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "useful": "grader",
        "not useful": "reformulate_question",
    },
)
# workflow.add_edge("reformulate_question", "generate")
workflow.add_edge("reformulate_question", "grader")
workflow.set_finish_point("grader")

app = workflow.compile()

In [128]:
inputs = [
    {
        "question": """"¿Cuáles son los algoritmos de busqueda encontrados con su correspondiente complejidad computacional?""",
        "rubric": """- Dos algoritmos con su Complejidad Computacional - 5 puntos
- Dos algoritmos - 4 puntos
- Un algoritmo con su Complejidad Computacional - 3 puntos
- Un algoritmo - 1 puntos
"""
    },
    {
        "question": """Basándote en la Base de Datos descrita en el documento proporcionado, ¿cuántas filas contiene?""",
        "rubric": """- número de filas mayor o igual a 5000 - 5 puntos
- número de filas mayor a 1000 - 4 puntos
- número de filas menor a 1000 - 3 puntos
- Utiliza la Base de Datos proporcionada en clase - 1 puntos
"""
    },
    {
      "question": """¿Qué "embeddings" se mencionan y cuál es su dimensionalidad?""",
      "rubric": """- grande y mediano - 5 puntos
- grande - 4 puntos
- mediano - 3 puntos
- pequeño - 1 puntos
"""  
    },
    {
        "question": """En el diseño experimental ¿Se menciona la cantidad de Querys o Queries, la cantidad de algoritmos y los casos bases?""",
        "rubric": """- Cantidad de Querys, Dos algoritmos y Dos casos base - 5 puntos
- Cantidad de Querys y Dos algoritmos - 4 puntos
- Cantidad de Querys, Un algoritmo y dos casos base - 3 puntos
- Un algoritmo y Un caso base - 1 puntos
        """
    },
    {
      "question": """En la evaluación de algoritmos de busqueda ¿Se menciona la cantidad de Querys o Queries, algoritmos de busqueda y casos bases?""",
      "rubric": """- Contiene cien Querys, dos algoritmos y dos casos base - 5 puntos
- Contiene cien Querys y dos algoritmos - 4 puntos
- Contiene diez Querys, un algoritmo y dos casos base - 3 puntos
- Contiene una Query, un algoritmo y un caso base - 1 puntos"""  
    },
    {
        "question": """"Según las conclusiones propuestas en el informe ¿Son estas son estadisticamente significativas y si están basadas en los resultados y marco teorico?""",
        "rubric": """- Establece conclusiones estadisticamente significativas - 5 puntos
- Establece conclusiones basadas en resultados y marco teórico - 4 puntos
- Establece conclusiones basadas en resultados - 3 puntos
- Establece conclusiones basadas en marco teórico - 1 puntos"""
    }
]

In [129]:
responses = app.batch(inputs)

---CHECK IF QUESTION IS COMPLEX---
---CHECK IF QUESTION IS COMPLEX---
---CHECK IF QUESTION IS COMPLEX---
---CHECK IF QUESTION IS COMPLEX---
---CHECK IF QUESTION IS COMPLEX---
---CHECK IF QUESTION IS COMPLEX---
score: si
---DECISION: QUESTION IS COMPLEX---
---COMPLEX GENERATE---
complex question: ¿Qué "embeddings" se mencionan y cuál es su dimensionalidad?
score: si
---DECISION: QUESTION IS COMPLEX---
---COMPLEX GENERATE---
complex question: "¿Cuáles son los algoritmos de busqueda encontrados con su correspondiente complejidad computacional?
score: no
---DECISION: QUESTION IS NOT COMPLEX---
---GENERATE---
question: Basándote en la Base de Datos descrita en el documento proporcionado, ¿cuántas filas contiene?
score: si
---DECISION: QUESTION IS COMPLEX---
---COMPLEX GENERATE---
complex question: En la evaluación de algoritmos de busqueda ¿Se menciona la cantidad de Querys o Queries, algoritmos de busqueda y casos bases?
score: si
---DECISION: QUESTION IS COMPLEX---
---COMPLEX GENERATE---


In [130]:
responses

[{'question': '"¿Cuáles son los algoritmos de busqueda encontrados con su correspondiente complejidad computacional?',
  'rubric': '- Dos algoritmos con su Complejidad Computacional - 5 puntos\n- Dos algoritmos - 4 puntos\n- Un algoritmo con su Complejidad Computacional - 3 puntos\n- Un algoritmo - 1 puntos\n',
  'generation': 'Lo siento, pero no hay información disponible sobre los algoritmos de búsqueda y su complejidad computacional en el contexto proporcionado. El contenido parece estar relacionado con la escalabilidad y precisión en búsquedas en bases de datos, pero no menciona específicamente algoritmos de búsqueda ni su complejidad.',
  'documents': [Document(metadata={'page': 4, 'source': './test/INF.pdf'}, page_content='escalabilidad de las búsquedas en bases de datos de gran tamaño, manteniendo al mismo tiempo \nuna alta precisión en los resultados obtenidos.'),
   Document(metadata={'page': 4, 'source': './test/INF.pdf'}, page_content='escalabilidad de las búsquedas en bases

In [132]:
for response in responses:
    print(response["answer"])

{'score': 1, 'feedback': 'La respuesta proporcionada no menciona específicamente dos algoritmos de búsqueda con su complejidad computacional, lo que es lo esperado en esta pregunta. Se sugiere buscar información adicional sobre este tema para mejorar la respuesta.'}
{'score': 1, 'feedback': 'La respuesta no utiliza la Base de Datos proporcionada, por lo que no puede determinar el número total de filas. La respuesta es correcta en cuanto a que no hay información sobre una Base de Datos proporcionada.'}
{'score': 4, 'feedback': 'La respuesta proporciona información sobre la herramienta "embeddings" utilizada en la búsqueda semántica, pero no menciona explícitamente su dimensionalidad. La respuesta cumple con lo esperado en cuanto a la descripción de la herramienta, pero falta información específica sobre la dimensionalidad.'}
{'score': 0, 'feedback': 'La respuesta no menciona la cantidad o tipo específico de queries realizadas, ni la cantidad de algoritmos utilizados en el diseño experim